In [1]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 131)

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix

from lightgbm import LGBMClassifier

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv("labels_curso - to_label_2.csv", index_col=0).dropna(subset=["y","watch-title"])

In [3]:
df = df[df['watch-time-text'].notnull()]
df = df[df['watch-time-text'].str.contains('horas')==False]
df.shape

(1274, 16)

In [4]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['watch-title']

## 1. Data Cleanner

In [5]:
clean_date = df['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)
#clean_date[1] = clean_date[1].map(lambda x: x[0].upper()+x[1:])

mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
clean_date.head()
df_limpo['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

## 2. Views Cleanner

In [6]:
views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo['views'] = views

## 3. Features

In [7]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()


In [8]:
features['publish_time'] = (pd.to_datetime('2020-06-01') - df_limpo['date']) / np.timedelta64(1,'D')
features['publish_time'] = features['publish_time'].replace(0,1)
features['views'] = df_limpo['views']
features['day_views'] = features['views'] / features['publish_time']
features = features.drop(['publish_time'], axis=1)

In [9]:
mask_train = df_limpo['date'] < "2019-09-01"
mask_val = df_limpo['date'] >= "2019-09-01"

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((636, 2), (638, 2), (636,), (638,))

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2)
#title_vec = TfidfVectorizer(min_df=5,ngram_range=(1,5))
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)


In [12]:
title_bow_train.shape

(636, 474)

In [13]:
from scipy.sparse import hstack, vstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [14]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((636, 476), (638, 476))

## 4. Random Forest

In [21]:
mdl_rf = RandomForestClassifier(n_estimators=1000, random_state=0, min_samples_leaf = 2, class_weight="balanced", n_jobs=6)
mdl_rf.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=6, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [112]:
p_rf = mdl_rf.predict_proba(Xval_wtitle)[:, 1]

In [113]:
average_precision_score(yval, p_rf), roc_auc_score(yval, p_rf)

(0.18187105538013426, 0.638463505412393)

## 5. LGBM

In [19]:
params = [0.027035241802913952, 1, 3, 0.33397697274506905, 0.3140136337076569, 802, 5, 5]
lr = params[0]
max_depth = params[1]
min_child_samples = params[2]
subsample = params[3]
colsample_bytree = params[4]
n_estimators = params[5]

min_df = params[6]
ngram_range = (1, params[7])

title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
#itle_vec = TfidfVectorizer(min_df=min_df)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

mdl_lgbm = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                     min_child_samples=min_child_samples, subsample=subsample,
                     colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                     class_weight="balanced", n_jobs=6)
mdl_lgbm.fit(Xtrain_wtitle, ytrain)

p_lgbm = mdl_lgbm.predict_proba(Xval_wtitle)[:, 1]


/home/rodrigorezendes/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [25]:


p_lgbm = mdl_lgbm.predict_proba(sparse.coo_matrix(Xval_wtitle))[:, 1]

NameError: name 'sparse' is not defined

In [115]:
average_precision_score(yval, p_lgbm), roc_auc_score(yval, p_lgbm)

(0.21420638006855222, 0.6133761400119329)

## 6. Logistic Reg

In [116]:
from sklearn.pipeline import make_pipeline

In [117]:
Xtrain_wtitle2 = csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = csr_matrix(Xval_wtitle.copy())

#scaler = StandardScaler()
#scaler = MaxAbsScaler()

#Xtrain_wtitle2[:, :2] = scaler.fit_transform(Xtrain_wtitle2[:, :2].todense())
#Xval_wtitle2[:, :2] = scaler.transform(Xval_wtitle2[:, :2].todense())
#Xtrain_wtitle2 = scaler.fit_transform(Xtrain_wtitle2)
#Xval_wtitle2 = scaler.transform(Xval_wtitle2)

lr_pipeline = make_pipeline(MaxAbsScaler(), LogisticRegression(C=0.5, penalty='l2',n_jobs=6, random_state=0))
lr_pipeline.fit(Xtrain_wtitle2, ytrain)

Pipeline(memory=None,
         steps=[('maxabsscaler', MaxAbsScaler(copy=True)),
                ('logisticregression',
                 LogisticRegression(C=0.5, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=6, penalty='l2',
                                    random_state=0, solver='lbfgs', tol=0.0001,
                                    verbose=0, warm_start=False))],
         verbose=False)

In [118]:
p_lr = lr_pipeline.predict_proba(Xval_wtitle2)[:, 1]

In [119]:
average_precision_score(yval, p_lr), roc_auc_score(yval, p_lr)

(0.18291003756069582, 0.5962155865556723)

## 7. Ensemble

In [120]:
p = (p_lr + p_rf + p_lgbm)/3
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.21087042246141888, 0.6339460749495697)

In [121]:
pd.DataFrame({"LR": p_lr, "RF": p_rf, "LGBM": p_lgbm}).corr()

,LR,RF,LGBM
LR,1.000000,0.682809,0.576687
RF,0.682809,1.000000,0.623111
LGBM,0.576687,0.623111,1.000000


In [122]:
p = 0.5*p_rf + 0.5*p_lgbm
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.2124296118552988, 0.6392590277580475)

## 8. Salva modelos

In [16]:
import joblib as jb

In [22]:
jb.dump(mdl_lgbm, "lgbm_20200617.pkl.z")
jb.dump(mdl_rf, "random_forest_20200617.pkl.z")
#jb.dump(lr_pipeline, "logistic_reg_20200208.pkl.z")
jb.dump(title_vec, "title_vectorizer_20200617.pkl.z")

['title_vectorizer_20200617.pkl.z']

In [23]:
jb.dump(title_vec, "title_vectorizer_20200617.pkl.z")

['title_vectorizer_20200617.pkl.z']